# **Installing BERTopic**

Run this notebook in Colab

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic
!pip install kaleido

#now this works:


In [ ]:
import pandas as pd
from bertopic import BERTopic
import kaleido #required
kaleido.__version__ #0.2.1
import plotly
plotly.__version__ #5.5.0
import plotly.graph_objects as go

# Data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

meta = pd.read_csv('/content/drive/MyDrive/6000k/data.csv')
#count how many has abstract
count = 0
index = []
for i in range(len(meta)):
    #print(i)
    
    if type(meta.iloc[i, 5])== float:
        count += 1
    else:
        index.append(i)
documents = meta.iloc[index,[1,2,6]]
documents=documents.reset_index()
documents["index"] = documents.index.values


In [ ]:
location = "Global"
for i in range(len(meta)):
  if location == "Global":
    break
  if documents.location[i] != location:
    documents = documents.drop(index = i,axis=0)
documents.shape

(8331, 4)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
documents.head()

,index,date,content,location
0,0,2020-02-14 03:04:33+00:00,work from home i m looking for stay at home mo...,New York
1,1,2020-02-04 23:21:24+00:00,limited time sign up for free ends march 31 wo...,New York
2,2,2020-03-31 21:21:25+00:00,catch our own tiffany joy murchison tonight vi...,New York
3,3,2020-03-31 21:18:46+00:00,clocking out is near make sure to end of your ...,New York
4,4,2020-03-31 20:04:07+00:00,she s so cool lea leainny stayhome goldenretri...,New York


In [ ]:
import re
documents.content = documents.apply(lambda row: re.sub(r"http\S+", "", row.content).lower(), 1)
documents.content = documents.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.content.split())), 1)
documents.content = documents.apply(lambda row: " ".join(re.sub("[^a-zA-Z0-9]+", " ", row.content).split()), 1)
stop = set(stopwords.words("english"))
documents.content = documents.apply(lambda row: " ".join(filter(lambda x:x[0] not in stop, row.content.split())), 1)
# documents.content = documents.apply(lambda row: " ".join([item for item in row.content.split() if item ]),1)
print(documents.head())
tweets = documents.content.to_list()
timestamps = documents.date.to_list()
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(tweets)

   index                       date  \
0      0  2020-02-14 03:04:33+00:00   
1      1  2020-02-04 23:21:24+00:00   
2      2  2020-03-31 21:21:25+00:00   
3      3  2020-03-31 21:18:46+00:00   
4      4  2020-03-31 20:04:07+00:00   

                                             content  location  
0  work from home looking for home limited up for...  New York  
1  limited up for free ends 31 work from home loo...  New York  
2  catch joy virtually literally hanging googleha...  New York  
3  clocking near end right how link wfh workingfr...  New York  
4  cool lea leainny goldenretriever wfh leadelase...  New York  


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Batches:   0%|          | 0/261 [00:00<?, ?it/s]

2022-05-18 02:08:43,761 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-05-18 02:09:45,304 - BERTopic - Reduced dimensionality
2022-05-18 02:09:46,404 - BERTopic - Clustered reduced embeddings


In [ ]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamps, nr_bins=20)

20it [00:04,  4.08it/s]


In [ ]:
a = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

a.update_layout(
    autosize=False,
    title=("Topics Over Time"+" ("+location+")"),
)
a.show()
a.write_image("/content/drive/MyDrive/6000k/nostopword/topics_over_time_"+location+'.png',format='png')

In [ ]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel

len(common_dictionary)

12

In [ ]:
a = len(topic_model.topic_names)
ll = []
for i in range(0,a-1):
    l = []
    topictmp = topic_model.get_topic(i)
    for j in topictmp:
      l.append(j[0])
    ll.append(l)
    
    # print(l)
   

# print(ll)

import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# Preprocess documents
cleaned_docs = topic_model._preprocess_text(tweets)

# Extract vectorizer and tokenizer from BERTopic
vectorizer = topic_model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [tokenizer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = ll

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='u_mass')
coherence = coherence_model.get_coherence()
print(coherence)

-11.340174955129738


In [ ]:
topic_model.get_representative_docs()

{0: ['when workingfromhome what',
  'workingfromhome beyond wonderful but godsent looking forward party',
  'foggy know will be find way work workingfromhome'],
 1: ['been craving chocolate bar w for now 200 calories per chocolate workingfromhome',
  'for box chocolate love wondered what working from home like pastry pastrychef chocolate workingfromhome chocolatecureseverything',
  'not which last lunch workingfromhome'],
 2: ['can we ban end from zoom zoommeetings workingfromhome',
  'letting public view private because covid19 wfh changing perceptions what private we resist protect nests we roll with line now gradations zoom',
  '61 had first ever zoom went better expected even now for walk covid19 keepingfit workingfromhome zoommeetings'],
 3: ['which combination leggings wear wfh',
  'workingfromhome wardrobe breaking but not footwear just covid19ontario',
  'realised literally wearing 5 wardrobe 95 comfy wfh workwear covidsucks nomakeupeither'],
 4: ['press lockdown baking with ex

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(tweets)

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [ ]:
freq = topic_model.get_topic_info();
freq.to_csv("/content/drive/MyDrive/6000k/nostopword/freq_"+location+".csv")

-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

[('workingfromhome', 0.12224404248626931),
 ('work', 0.06289239538275739),
 ('for', 0.05307136659694194),
 ('working', 0.046605435104105225),
 ('wfh', 0.0452364069735414),
 ('home', 0.0405584172989662),
 ('coffee', 0.03968549914497854),
 ('from', 0.03937906163684534),
 ('not', 0.03870301254323531),
 ('with', 0.036259218034110766)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [ ]:
topic_model.visualize_distribution(probs[100], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=12)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
b = topic_model.visualize_barchart(top_n_topics=12)

b.update_layout(
    autosize=False,
    title=("Topics Word Scores"+" ("+location+")"),
)
b.show()
b.write_image("/content/drive/MyDrive/6000k/nostopword/"+"topic_word_scores_"+location+".png")

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=10, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [ ]:
topic_model.update_topics(tweets, topics, n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
new_topics, new_probs = topic_model.reduce_topics(tweets, topics, probs, nr_topics=60)

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [ ]:
similar_topics, similarity = topic_model.find_topics("food", top_n=5); similar_topics

[2, 8, 6, 56, 13]

In [ ]:
topic_model.get_topic(2)

[('lunch', 0.024133588349627234),
 ('breakfast', 0.012646975841215647),
 ('and', 0.012146086839072165),
 ('wfh', 0.011439243673503848),
 ('with', 0.010594776200100733),
 ('for', 0.010390998875604934),
 ('of', 0.009669050194786857),
 ('food', 0.009576746430649097),
 ('the', 0.00922208200266935),
 ('to', 0.008955122393540241)]

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
